In [ ]:
!pip install tensorflow_text
!pip install tensorflow_addons
!pip install transformers

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import pickle
import numpy as np
import pandas as pd
import tensorflow_addons as tfa

In [ ]:
gpus = tf.config.list_physical_devices('GPU')

In [ ]:
gpus

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [ ]:
df = pd.read_csv("DesireDBPreprocessed.csv")
df.head(5)

,Unnamed: 0,Content,Label
0,0,"Ah, it's the weekend again. This has become a ...",Fulfilled
1,1,It was hectic. Then on top of that the one and...,Unfulfilled
2,2,"Being a groomsman, I really didn't get the cha...",Fulfilled
3,3,Before justice started school I had an idea to...,Fulfilled
4,4,So for some reason (I dunno wad) I was under t...,Unfulfilled


In [ ]:
df.groupby('Label').describe()

Unnamed: 0                            ...                         
                 count         mean          std  ...     50%      75%     max
Label                                             ...                         
Fulfilled       1950.0  1915.909744  1052.956578  ...  1968.5  2854.75  3586.0
Unfulfilled     1126.0  1633.002664   946.576897  ...  1590.5  2365.75  3587.0
Unknown          512.0  1680.259766  1097.551201  ...  1618.5  2730.50  3584.0

[3 rows x 8 columns]

In [ ]:
df['Label'].value_counts()

Fulfilled      1950
Unfulfilled    1126
Unknown         512
Name: Label, dtype: int64

In [ ]:
X = df.iloc[:, 1 :-1].values
y = df.iloc[:, -1].values

In [ ]:
LABELS = [
    'admiration',
    'amusement',
    'anger',
    'annoyance',
    'approval',
    'caring',
    'confusion',
    'curiosity',
    'desire',
    'disappointment',
    'disapproval',
    'disgust',
    'embarrassment',
    'excitement',
    'fear',
    'gratitude',
    'grief',
    'joy',
    'love',
    'nervousness',
    'optimism',
    'pride',
    'realization',
    'relief',
    'remorse',
    'sadness',
    'surprise',
    'neutral',
]

In [ ]:
def values_to_label(values):

  maxim = -1 
  index = -1

  for i in range(len(values)):
    if values[i] > maxim: 
      maxim = values[i] 
      index = i

  # print(LABELS[index])

  return LABELS[index]


In [ ]:
X_with_emotion = []

In [ ]:
with open('out_visual.pickle', 'rb') as fd:
    w = pickle.load(fd)

    for i in range(len(X)):

      label = values_to_label(w[i][1])
      x_simple = X[i].tolist()
      x_simple[0] = x_simple[0] + " " + label
      X_with_emotion.append(x_simple)


In [ ]:
markers = {"accordingly": 0, "so": 0,"ultimately": 0,"finally": 0, "rather": 0, "yet": 0, "although": 0, "but": 0}

In [ ]:
for story in X_with_emotion:
  word_list = story[0].split()
  for word in word_list:
    if word in markers:
      markers[word] += 1
  story[0] += " "+str(sum(list(markers.values())[:4]))
  story[0] += " "+str(sum(list(markers.values())[-4:]))

  markers = {k: 0 for k in markers}

In [ ]:
markers

{'accordingly': 0,
 'although': 0,
 'but': 0,
 'finally': 0,
 'rather': 0,
 'so': 0,
 'ultimately': 0,
 'yet': 0}

In [ ]:
X_with_emotion[:2]

[["Ah, it's the weekend again. This has become a sort of weekend blog, hasn't it? For Saturday morning's activity, I decided to organize a small team for trekking the nice and simple route of Kranji Memorial trek once again. However, as the weather gods would have it, I woke up to ominous looking skies. Soon, it started pouring and one by one, people decided to back out. Finally, only Jane, Felix, Jervais and I were left. But hey, the weather turned out to be really awsome! Raving to go, people! neutral 0 0"],
 ["It was hectic. Then on top of that the one and ONLY day I have ever worn a skirt and pantyhose to work, I had to spend the afternoon crawling on the floor under my desk plugging shit in. I am not a dressy person, I don't like dresses and skirts. I would much rather wear pants, and I certainly do not like pantyhoes or stockings of any kind. But I live in the south, and ladies are expected to wear them if they are showing any leg at all, it is just not socially acceptable in my 

In [ ]:
X_with_emotion = [x for x,label in zip(X_with_emotion,y) if label =='Fulfilled' or label =='Unfulfilled']
y = [label for label in y if label =='Fulfilled' or label =='Unfulfilled']

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_with_emotion,y,test_size=0.2, random_state=42)

In [ ]:
len(X_train)

2460

In [ ]:
label_to_id= {'Fulfilled' : 0, 'Unfulfilled' : 1}

In [ ]:
y_train = [label_to_id[label] for label in y_train]

In [ ]:
y_train = tf.one_hot(y_train, depth = 2)

In [ ]:
y_test = [label_to_id[label] for label in y_test]
y_test = tf.one_hot(y_test, depth = 2)

In [ ]:
y_train

<tf.Tensor: shape=(2460, 2), dtype=float32, numpy=
array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [0., 1.],
       [0., 1.],
       [0., 1.]], dtype=float32)>

In [ ]:
tf.random.set_seed(0)

In [ ]:
def batch_encode(tokenizer, texts, batch_size=256):
    input_ids = []
    attention_masks = []
    for x in texts:
      x = tokenizer(x, return_tensors='tf', truncation=True, padding=True, pad_to_multiple_of=512)  # to cut sentences with > 512 words
      input_ids.append(x['input_ids'])
      attention_masks.append(x['attention_mask'])

    # return tf.convert_to_tensor(h), tf.convert_to_tensor(np.array(attention_masks))
    return tf.convert_to_tensor(input_ids), tf.convert_to_tensor(attention_masks)
    
    

In [ ]:
from transformers import  DistilBertTokenizer, DistilBertModel
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Encode X_train
X_train_ids, X_train_attention = batch_encode(tokenizer, X_train)

# Encode X_test
X_test_ids, X_test_attention = batch_encode(tokenizer, X_test)

In [ ]:
print(X_train_ids[0])
print(X_train_attention[0])
print(X_test_ids[0])
print(X_test_attention[0])

tf.Tensor(
[[  101  2002  2001 15261  2012  1996  4511  1010  1996  2833  1010  1996
   8012  1997  1996 12846  2008  2134  2102  3599  5574  2000  2026  5510
   8569  5104  1010  2021  2009  2428  2106  2007  2010  1012  2673  1045
   3740  2001  2005  2032  1010  2009  2790  2130  2026  3745  1997  2833
   2036  2001  2005  2032  1012  5292  3270  1012  2061  2172  2005  2771
   1010  1045  2347  2102  8510  2007  1996  2833  1045  2245  1997 13063
  26666  2000  2191  2870  3407  1012  2021  1045  2134  2102  1012  2044
   4596  1010  2904  1999  2000  2026 20345  1010  1998  2253 14855 10841
  13793  2007 14006  1010  1998  2062 14006  2008  2057  1031  7303  1033
   2005  2138  1045  2001  4452  1997  1996  2601  1012  1045  4711  3123
  15829  1999  1996  2300  1012  2009  2001  2428 11895  6559  1012  5292
   3270  1012  2064  2102  6235  2009  2012  2305  1012  2049  2061  2367
   1999  1996  2154  1012  6298  3325  2008  3383  2028  2154  1045  2097
   5293  2138  1045  3685  

In [ ]:
METRICS = [
      'accuracy',
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

In [ ]:
def unfreeze_model(model):
    # Used for fine-tuning the model
    for layer in model.layers[-100:]:
        layer.trainable = True

    return model

In [ ]:
from transformers import TFDistilBertModel, DistilBertConfig

DISTILBERT_DROPOUT = 0.2
DISTILBERT_ATT_DROPOUT = 0.2
 
# Configure DistilBERT's initialization
config = DistilBertConfig(dropout=DISTILBERT_DROPOUT, 
                          attention_dropout=DISTILBERT_ATT_DROPOUT, 
                          output_hidden_states=True)

LAYER_DROPOUT = 0.2
LEARNING_RATE = 3e-5
RANDOM_STATE = 1

init_lr = 5e-3
# # optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    init_lr,
    decay_steps=300,
    decay_rate=0.75,
    staircase=True)
# optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

loss = tf.keras.losses.BinaryCrossentropy()

transformer_model = TFDistilBertModel.from_pretrained('distilbert-base-uncased', config=config)
transformer_model.trainable = False
transformer_model = unfreeze_model(transformer_model)

weight_initializer = tf.keras.initializers.GlorotNormal(seed=RANDOM_STATE)

def build_classifier_model_ft(transformer):
  input_ids_layer = tf.keras.layers.Input(shape=(512,), 
                                            name='input_ids', 
                                            dtype='int32')
  input_attention_layer = tf.keras.layers.Input(shape=(512,), 
                                                  name='input_attention', 
                                                  dtype='int32')
    
  last_hidden_state = transformer([input_ids_layer, input_attention_layer])[0]
  
  cls_token = last_hidden_state[:, 0, :]

  output = tf.keras.layers.Dense(2, activation='sigmoid', 
                                 kernel_initializer=weight_initializer,  kernel_constraint=None,
                                 bias_initializer='zeros')(cls_token)

  model = tf.keras.Model([input_ids_layer, input_attention_layer], output)
  model.compile(tf.keras.optimizers.Adam(learning_rate=lr_schedule), loss=loss, metrics=METRICS)

  return model

model = build_classifier_model_ft(transformer_model)

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertModel: ['vocab_transform', 'vocab_projector', 'vocab_layer_norm', 'activation_13']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


In [ ]:
X_train_ids_1 = tf.convert_to_tensor([o[0] for o in X_train_ids])
X_train_attention_1 = tf.convert_to_tensor([o[0] for o in X_train_attention])
X_test_ids_1 = tf.convert_to_tensor([o[0] for o in X_test_ids])
X_test_attention_1 = tf.convert_to_tensor([o[0] for o in X_test_attention])

In [ ]:
EPOCHS = 35
BATCH_SIZE = 32 
# 32
model.summary()
print(len(X_train_ids_1))
print(len(X_train_attention_1))
# Train the model
train_history1 = model.fit(
    x = (X_train_ids_1, X_train_attention_1),
    y = y_train,
    epochs = EPOCHS,
    batch_size = BATCH_SIZE,
    # validation_data = ((X_test_ids, X_test_attention), y_test),
    verbose=1
)

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 512)]        0           []                               
                                                                                                  
 input_attention (InputLayer)   [(None, 512)]        0           []                               
                                                                                                  
 tf_distil_bert_model_4 (TFDist  TFBaseModelOutput(l  66362880   ['input_ids[0][0]',              
 ilBertModel)                   ast_hidden_state=(N               'input_attention[0][0]']        
                                one, 512, 768),                                                   
                                 hidden_states=((No                                         

In [ ]:
model.save_weights('distilbert1')
# model.load_weights('distilbert1')


model.evaluate((X_test_ids_1, X_test_attention_1), y_test)

20/20 [==============================] - 23s 1s/step - loss: 0.5890 - accuracy: 0.7078 - precision: 0.7076 - recall: 0.7110


[0.589024007320404, 0.7077922224998474, 0.7075929045677185, 0.7110389471054077]

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score, f1_score, precision_score,recall_score

y_predicted = model.predict((X_test_ids_1, X_test_attention_1))
y_predicted_modeled = tf.math.argmax(y_predicted,axis=1)
print(np.unique(y_predicted_modeled))
y_test_modeled =  tf.math.argmax(y_test,axis=1)

cm = confusion_matrix(y_test_modeled, y_predicted_modeled)
print(cm)


f1=f1_score(y_true= y_test_modeled, y_pred= y_predicted_modeled)

#Print the accuracy
print(f1)
print(accuracy_score(y_true= y_test_modeled, y_pred= y_predicted_modeled))
print(precision_score(y_true= y_test_modeled, y_pred= y_predicted_modeled, average='micro'))
print(recall_score(y_true= y_test_modeled, y_pred= y_predicted_modeled, average='micro'))
print(f1_score(y_true= y_test_modeled, y_pred= y_predicted_modeled,average='micro'))

[0 1]
[[358  38]
 [142  78]]
0.4642857142857143
0.7077922077922078
0.7077922077922078
0.7077922077922078
0.7077922077922078
